Importing primary dependencies for data fetch and cleanup

In [ ]:
import pandas as pd
import numpy as np
import re

from pandas import read_csv

Various constants and data fetch

In [ ]:
max_features=3600
max_len=1024
test_prop=0.3
epoch_count=20
batch_size=2048
fake=read_csv('https://raw.githubusercontent.com/MobiusYeeitus/CSProj/master/datasets/set1/fake.csv')
real=read_csv('https://raw.githubusercontent.com/MobiusYeeitus/CSProj/master/datasets/set1/true.csv')



In [ ]:
#fake.sample(4)

In [ ]:
#real.sample(4)

Compressed every cleanup job into the *clean_csv* function

In [ ]:
def clean_string(string): #function to clean a string | to be used in clean_csv
    string=string.lower() #convert string to lowercase
    string=re.sub('\n|\r|\t','',string) #cleaning of whitespaces
    string=re.sub(r'[^\w\s]+','',string) #cleaning of symbols
    return string
def clean_csv(df,identity): #cleaning the CSV file
    print('Cleaning CSV.')
    df=df.dropna() #drop all tuples with empty attributes
    length=[]
    [length.append(len(str(text))) for text in df["text"]] #calculate length of an article
    df['length'] = length #saves the length of the artivle in a new column
    df = df.drop(df['text'][df['length'] < 120].index, axis = 0) #drop all tuples with less than 120 letters
    df['text']=df['title']+" "+df['text'] #append text to title
    vfunc=np.vectorize(clean_string) #vectorize th clean_string function for use in pandas df
    df['text']=vfunc(df['text']) #apply the vectorized function to further clean the data
    df['identity']=identity #add the identity (T/F) or (0/1) in a new attribute to the data
    df.reset_index() #reset all indexes of the dataset
    print('Cleaning completed.')
    return df

In [ ]:
fake=clean_csv(fake,0)

In [ ]:
#fake.sample(4)

In [ ]:
real=clean_csv(real,1)

In [ ]:
#real.sample(4)

Dropping all unnecessary columns

In [ ]:
fake=fake[['text','identity']]
real=real[['text','identity']]

In [ ]:
#fake.sample(2)

In [ ]:
data=pd.concat([real,fake])

In [ ]:
data.sample(4)

In [ ]:
data.to_csv('datasets/output/cache/experimental-branch-cache/ds1.csv')

Importing dependencies for ANN and Representation

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.models import Sequential
from keras.layers import Dense,Embedding,LSTM,Conv1D,MaxPool1D,Dropout,GRU
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,accuracy_score

Tokenizing the dataset for use in the ANN

In [ ]:
def tokenize_df(df,max_features): #tokenize the df using keras tokenizer
        print('Tokenizing DF.')
        tokenizer=Tokenizer(num_words=max_features,filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',split =' ') #filters is redundant, but still keeping if I missed any symbols previously
        tokenizer.fit_on_texts(texts=df['text']) #fit
        X = tokenizer.texts_to_sequences(texts=df['text'])
        X = pad_sequences(sequences=X,maxlen=max_len,padding='pre') #padding the set of words
        Y = df['identity'].values #identities (0/1)
        V=tokenizer.word_index #vocabulary
        print('Tokenized DF.')
        return X,Y,V

In [ ]:
X,Y,V=tokenize_df(data,max_features)

Creating the model and compiling it to be used.

In [ ]:
def create_lstm_model(max_features): #creating the internal structure of the simple LSTM model
        print('Creating LSTM model.')
        lstm_model=Sequential(name='Sierra')
        lstm_model.add(layer=Embedding(input_dim=128,output_dim=128,name='Alpha'))
        lstm_model.add(layer=LSTM(units=128,name='Lima'))
        lstm_model.add(layer=Dense(1,activation='sigmoid',name ='Omega'))
        print('Model creation completed.')
        lstm_model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
        print('Model compiled.')
        return lstm_model

In [ ]:
lstm_model=create_lstm_model(max_features)

In [ ]:
X_train,X_text,Y_train,Y_test=train_test_split(X,Y)

In [ ]:
lstm_model.fit(X_train,Y_train,batch_size=batch_size,epochs=epoch_count,verbose=1,validation_split=test_prop,shuffle=1,use_multiprocessing=True)

In [ ]:
Y_pred = (lstm_model.predict(X_text) >=0.5).astype(int)

In [ ]:
accuracy_score(Y_test, Y_pred)

In [ ]:
print(classification_report(Y_test, Y_pred))

In [ ]:
tokenizer = Tokenizer(num_words = max_features, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', split = ' ')
x = ["Vladimir Putin and Russia are not the aggressors in this conflict, and that the U.S. orchestrated the 2022 invasion of Ukraine."]
x = tokenizer.texts_to_sequences(x)
x = pad_sequences(x, maxlen=max_len)

In [ ]:
l = [(lstm_model.predict(x) >=0.5).astype(int)]
if l[0][0]==0:
  print('FAKE')
else:
  print('REAL')

In [ ]:
lstm_model.save('models/new')

In [ ]:
nmod=tf.keras.saving.load_model('models/new')

In [ ]:
y_pred = (nmod.predict(X_text) >=0.5).astype(int)